In [25]:
from pyspark.sql import SparkSession
import sys
spark = SparkSession.builder.getOrCreate()
from pymongo import MongoClient
import hdfs
from pywebhdfs.webhdfs import PyWebHdfsClient
from pprint import pprint
import json
from datetime import datetime as dt
import pandas as pd

In [29]:
hdfs = PyWebHdfsClient(host='hadoop-master',port='9870', user_name='kafka-data')  # your Namenode IP & username here
my_dir = '/user/kafka-data'
data_files = [i["pathSuffix"] for i in hdfs.list_dir(my_dir)["FileStatuses"]["FileStatus"]]
pprint(data_files)

['data-0.json',
 'data-1.json',
 'data-10.json',
 'data-11.json',
 'data-12.json',
 'data-13.json',
 'data-14.json',
 'data-15.json',
 'data-16.json',
 'data-17.json',
 'data-18.json',
 'data-19.json',
 'data-2.json',
 'data-20.json',
 'data-21.json',
 'data-22.json',
 'data-23.json',
 'data-24.json',
 'data-25.json',
 'data-3.json',
 'data-4.json',
 'data-5.json',
 'data-6.json',
 'data-7.json',
 'data-8.json',
 'data-9.json']


In [4]:
conn = MongoClient('mongo', 27017)
db = conn.tweets
collection = db.tweet

In [30]:
collection.drop()

for i in data_files:
    df = spark.read.json(f'hdfs://hadoop-master:9000/user/kafka-data/{i}')
    json_data = df.toJSON().collect()
    for i in json_data:
        data = json.loads(i)
        hashtags = [i for i in data["hashtags"]]
        for h in hashtags:
            new_datetime = dt.utcfromtimestamp(int(data["timestamp"])/1000)
            uniqued_hashtag = {"hashtag": h, "datetime": new_datetime}
            collection.insert_one(uniqued_hashtag)
        
cursor = collection.find()
for record in cursor:
    print(record)

{'_id': ObjectId('60d5a10be0a097dedfda509a'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 9, 23, 39, 878000)}
{'_id': ObjectId('60d5a10be0a097dedfda509b'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 9, 23, 40, 86000)}
{'_id': ObjectId('60d5a10be0a097dedfda509c'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 9, 23, 40, 374000)}
{'_id': ObjectId('60d5a10be0a097dedfda509d'), 'hashtag': 'NazisrausausdenStadien', 'datetime': datetime.datetime(2021, 6, 25, 9, 23, 40, 537000)}
{'_id': ObjectId('60d5a10be0a097dedfda509e'), 'hashtag': 'HUNGER', 'datetime': datetime.datetime(2021, 6, 25, 9, 23, 40, 537000)}
{'_id': ObjectId('60d5a10be0a097dedfda509f'), 'hashtag': 'Goretzka', 'datetime': datetime.datetime(2021, 6, 25, 9, 23, 40, 537000)}
{'_id': ObjectId('60d5a10be0a097dedfda50a0'), 'hashtag': 'EURO2020', 'datetime': datetime.datetime(2021, 6, 25, 9, 23, 40, 537000)}
{'_id': ObjectId('60d5a10be0a097dedfda50a1'), 'hashtag': 'Euro202

{'_id': ObjectId('60d5a10de0a097dedfda569f'), 'hashtag': 'homesforsale', 'datetime': datetime.datetime(2021, 6, 24, 22, 59, 30, 452000)}
{'_id': ObjectId('60d5a10de0a097dedfda56a0'), 'hashtag': '24Jun', 'datetime': datetime.datetime(2021, 6, 24, 22, 59, 30, 639000)}
{'_id': ObjectId('60d5a10de0a097dedfda56a1'), 'hashtag': 'PNM', 'datetime': datetime.datetime(2021, 6, 24, 22, 59, 30, 639000)}
{'_id': ObjectId('60d5a10de0a097dedfda56a2'), 'hashtag': 'MUFC', 'datetime': datetime.datetime(2021, 6, 24, 22, 59, 30, 601000)}
{'_id': ObjectId('60d5a10de0a097dedfda56a3'), 'hashtag': '越乃煎餅', 'datetime': datetime.datetime(2021, 6, 24, 22, 59, 30, 598000)}
{'_id': ObjectId('60d5a10de0a097dedfda56a4'), 'hashtag': 'A賞', 'datetime': datetime.datetime(2021, 6, 24, 22, 59, 30, 598000)}
{'_id': ObjectId('60d5a10de0a097dedfda56a5'), 'hashtag': '越乃煎餅新発売', 'datetime': datetime.datetime(2021, 6, 24, 22, 59, 30, 598000)}
{'_id': ObjectId('60d5a10de0a097dedfda56a6'), 'hashtag': 'liarfromtheshire', 'datetime':